In [274]:
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.layers import Reshape, MaxPooling2D
from tensorflow.keras.layers import InputLayer, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from concurrent.futures import ThreadPoolExecutor
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2 as cv
import os

In [275]:
tf.__version__

'2.15.0'

In [276]:
video_path = '../data/test/test_video.mp4'  # Change this to the path of your video file

In [277]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.config.list_physical_devices('GPU'):
    print("GPU is detected.")
else:
    print("GPU not detected, check installation.")

Num GPUs Available:  1
GPU is detected.


In [278]:
# List the devices TensorFlow can use
print("Available devices:")
print(tf.config.list_physical_devices())

# Check if a GPU is available
print("Is a GPU available: ", tf.test.is_gpu_available())

Available devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is a GPU available:  True


2024-05-19 01:43:11.178062: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-19 01:43:11.178287: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-19 01:43:11.178368: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [279]:
# Load the ML model
model = tf.keras.models.load_model('../models/model3.keras')

In [280]:
# Get webcam stream
cap = cv.VideoCapture(video_path)

In [281]:
num_contours = 25
desired_dim = (150, 150)  # Example desired dimension

In [282]:
tf.debugging.set_log_device_placement(True)
tf.device('/GPU:0')

In [283]:
# Load mean and standard deviation
mean = np.load('../models/mean.npy')
std = np.load('../models/std.npy')

In [284]:
while True:
    # Read a frame from the camera
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for contour detection
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    gray = cv.GaussianBlur(gray, (5, 5), 0)
    
    # Apply thresholding
    _, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    # Find contours
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    # Process only up to `num_contours` (using Python slicing)
    for cnt in contours[:num_contours]:
        # Get the bounding rectangle for the contour
        x, y, w, h = cv.boundingRect(cnt)

        # Crop and resize the image
        cropped_image = gray[y:y + h, x:x + w]
        resized_image = cv.resize(cropped_image, desired_dim)
        reshaped_image = (np.array(resized_image).astype('float32') - mean) / std
        reshaped_image = reshaped_image.reshape(-1, 150, 150, 1)

        # Draw the current contour on the original frame if the model predicts it as true
        if model.predict(reshaped_image, verbose = False)[0][1] > 0.90:
            cv.drawContours(frame, [cnt], 0, (0, 255, 0), 3)
        

    # Display the frame with drawn contours
    cv.imshow('frame', frame)

    # Break the loop on pressing 'q'
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and destroy all windows
cap.release()
cv.destroyAllWindows()